In [30]:
import csv
import requests
import json
import math

# Importing the PIL library
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

from statistics import mean
from scipy.stats import binom
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import matplotlib.patches as mpatches
import seaborn as sns
pd.options.mode.chained_assignment = None  # default='warn'

xG_matrix = [
    
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 16.67, 16.67, 16.67, 40.0, 0.0, 22.22, 54.55, 0.0, 25.0, 0.0, 0.0],
    [0.0, 21.43, 16.67, 16.67, 11.88, 41.79, 60.71, 31.39, 10.25, 10.39, 0.0, 7.89, 0.0],
    [3.0, 4.0, 13.24, 13.9, 19.26, 38.28, 42.19, 31.88, 15.53, 12.96, 8.57, 4.29, 3.0],
    [4.0, 7.14, 12.7, 14.29, 17.87, 26.11, 28.17, 23.4, 13.77, 11.86, 8.72, 6.75, 4.0],
    [3.45, 7.51, 9.2, 11.76, 16.45, 21.8, 24.47, 19.66, 14.41, 9.27, 6.43, 5.71, 5.56],
    [5.86, 6.29, 8.86, 11.95, 15.53, 18.56, 20.68, 15.78, 11.88, 9.12, 6.01, 6.65, 5.0],
    [5.03, 6.9, 7.48, 9.75, 13.65, 16.0, 15.29, 13.7, 8.85, 6.38, 6.25, 7.56, 4.55],
    [4.02, 6.16, 6.15, 9.64, 8.85, 12.64, 12.73, 10.48, 8.68, 5.59, 5.29, 5.33, 5.41],
    [3.16, 4.88, 5.34, 5.56, 7.28, 9.76, 9.65, 9.82, 6.25, 6.18, 3.52, 6.16, 5.5],
    [2.69, 4.91, 4.57, 5.44, 5.7, 7.35, 7.15, 6.38, 5.29, 4.39, 2.5, 5.76, 4.33],
    [1.89, 2.41, 3.3, 3.55, 4.77, 5.24, 5.28, 5.98, 5.32, 4.31, 3.84, 3.93, 3.33],
    [1.86, 3.33, 1.29, 3.27, 3.31, 5.03, 5.74, 4.76, 3.2, 2.83, 4.64, 4.55, 2.0],
    [1.56, 1.98, 2.72, 2.32, 3.88, 4.38, 4.9, 4.03, 2.67, 2.05, 2.39, 1.22, 1.09],

    ];

xGOT_matrix = [
    
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
    [0.0, 0.0, 25.0, 0.0, 16.67, 66.67, 0.0, 28.57, 60.0, 0.0, 25.0, 0.0, 0.0], 
    [4.0, 10.08, 16.67, 17.0, 18.46, 46.06, 62.96, 38.45, 14.62, 16.0, 12.0, 9.38, 4.0], 
    [4.0, 5.56, 18.75, 20.97, 27.52, 43.92, 46.41, 37.84, 22.97, 21.78, 16.54, 6.67, 4.0], 
    [5.0, 13.79, 23.13, 23.19, 26.32, 31.98, 33.8, 30.16, 21.49, 21.49, 18.52, 15.49, 6.0], 
    [9.09, 15.83, 18.09, 20.53, 25.98, 30.73, 33.88, 29.69, 24.7, 18.95, 13.52, 12.93, 10.29], 
    [12.0, 12.91, 18.07, 22.28, 27.26, 30.6, 32.72, 28.78, 24.11, 20.1, 14.47, 16.13, 12.0], 
    [12.28, 16.48, 17.2, 20.85, 27.9, 31.02, 29.25, 29.08, 21.58, 16.47, 16.17, 18.05, 9.09], 
    [14.75, 16.4, 16.47, 24.29, 22.33, 30.22, 28.02, 27.85, 24.12, 16.49, 15.16, 14.07, 13.33], 
    [10.14, 13.18, 15.24, 16.27, 21.33, 25.98, 25.43, 27.54, 20.81, 18.95, 10.76, 14.97, 12.53], 
    [8.21, 14.01, 14.16, 17.5, 18.62, 22.34, 22.07, 20.78, 18.26, 14.04, 12.86, 12.67, 8.33], 
    [5.76, 7.32, 10.56, 12.7, 15.67, 17.67, 17.12, 20.0, 16.59, 13.64, 12.59, 7.02, 4.33], 
    [4.33, 7.81, 3.88, 10.45, 10.26, 14.56, 16.72, 14.69, 9.32, 11.11, 12.5, 10.0, 7.0], 
    [4.09, 10.43, 11.11, 14.46, 14.29, 15.25, 14.69, 11.0, 10.76, 9.58, 9.33, 10.33, 4.29]

    ];

maxX = 1700 # Arvioitu, päätyviiva 0 - keskiviiva 1700
maxY = 2000 # [-1000, 1000], maalivahdin näkökulmasta katsottuna oikealle negatiivinen, 0 keskilinjalla

def calcxG(x,y):

    y = 1000 + y
    
    if x > maxX:
        x = maxX-1

    xd = 2 + math.floor(x / maxX * 12)
    yd = math.floor(y / maxY * 13)
    xGOT = xGOT_matrix[xd][yd] / 100
    xG = xG_matrix[xd][yd] / 100
    return (xGOT, xG)

In [49]:

api_key = 'n76qrhjnyygtcz7fzhg57sftbv6wtgjk'
gameID = '716369'
totxGF = 0
totxGA = 0
nGames = 0
teamn = "Indians"

url = 'https://salibandy.api.torneopal.com/taso/rest/getMatch?api_key='+api_key+'&match_id='+gameID
response = requests.get(url)
temp = response.json()
a = temp['match']
events = a['events']
pd_events = pd.DataFrame(events)
pd_events = pd_events.drop(['note','time_min','time_sec','lineup_id', 'lineup_id_2', 'player_2_id','kotivieras', 'player_2_name', 'wall_time', 'code_en','code_sv', 'connected_event_id', 's_A', 's_B'], axis=1)
shots = pd_events[((pd_events['code'] == "laukausohi") | (pd_events['code'] == "laukausblokattu") | (pd_events['code'] == "laukausmaali") | (pd_events['code'] == "laukaus"))]
shots = shots.reset_index(drop=True)
shots['xGOT'] = 0
shots['xG'] = 0
team1name = a['team_A_name']
team2name = a['team_B_name']

for i in range(len(shots)):
    st = shots['location'].iloc[i].split(',')
    x = float(st[1])
    y = float(st[0])
    (xGOT, xG) = calcxG(x,y)
    if shots['code'].iloc[i] == 'laukaus' or shots['code'].iloc[i] == 'laukausmaali':
        shots.loc[i,'xGOT'] = xGOT
    else: shots.loc[i,'xGOT'] = 0
    shots.loc[i,'xG'] = xG

team1 = shots[shots['team'] == "A"]
team2 = shots[shots['team'] == "B"]

goals = shots[shots['code'] == "laukausmaali"]
goalst1 = goals[goals['team'] == "A"]
goalst2 = goals[goals['team'] == "B"]

team1xG = np.round(team1['xG'].sum(),decimals=2)
team2xG = np.round(team2['xG'].sum(),decimals=2)

team1xGOT = np.round(team1['xGOT'].sum(),decimals=2)
team2xGOT = np.round(team2['xGOT'].sum(),decimals=2)

team1go = len(goalst1)
team2go = len(goalst2)

print(team1name + " | Goals: " + str(team1go) + " | xG: " + str(team1xG) + " | xGOT: " + str(team1xGOT) + " | Shots: " + str(len(team1)))
print(team2name + " | Goals: " + str(team2go) + " | xG: " + str(team2xG) + " | xGOT: " + str(team2xGOT) + " | Shots: " + str(len(team2)))

Rangers | Goals: 10 | xG: 4.57 | xGOT: 5.48 | Shots: 46
M-Team | Goals: 5 | xG: 4.86 | xGOT: 5.52 | Shots: 47


In [51]:
api_key = 'n76qrhjnyygtcz7fzhg57sftbv6wtgjk'

url = 'https://salibandy.api.torneopal.com/taso/rest/getMatches?api_key='+api_key+'&season_id=2023-2024&competition_id=sb2023&category_id=402&group_id=2'
response = requests.get(url)
temp = response.json()
#/taso/rest/getMatches?{api_key,season_id,competition_id,category_id,group_id,venue_id,joined_venues,venue_city,date,team_id,start_date,end_date,referee_id,club_id,details,category_group,match_number,match_status,match_id,home_away,find,per_page,page,page_size,page_number,official_only,competition_officiality
matches_json = temp['matches']
matches = pd.DataFrame(matches_json)
temp

{'call': {'method': 'getMatches',
  'parameters': {'competition_id': 'sb2023',
   'category_id': '402',
   'group_id': '2',
   'season': '2023-2024'},
  'per_page': 0,
  'page': 1,
  'total_result_count': 198,
  'status': 'ok',
  'result_count': 198,
  'result_time': 0.263,
  'time_stamp': 1699212428},
 'matches': [{'match_id': '715140',
   'match_number': '1359',
   'match_report': '',
   'match_external_id': '',
   'age_group': '',
   'aggregate_match': '',
   'competition_id': 'sb2023',
   'competition_name': 'Valtakunnalliset sarjat 2023-24',
   'competition_result_service_name': '',
   'competition_logo': '',
   'competition_status': 'published',
   'competition_officiality': 'official',
   'season_id': '2023-2024',
   'organiser': 'ssbl',
   'organiser_private': '0',
   'category_id': '402',
   'category_external_id': '',
   'category_abbrevation': '402',
   'category_name': 'F-liiga miehet',
   'category_logo': 'https://storage.googleapis.com/client-salibandyliiga-prod/2020/03/6

In [50]:
temp

{'call': {'method': 'getMatch',
  'status': 'ok',
  'access': [],
  'timestamp': '2023-11-05T21:22:23',
  'result_time': 0.605,
  'time_stamp': 1699212143},
 'match': {'match_id': '716369',
  'match_number': '1700',
  'match_report': 'https://salibandy.api.torneopal.com/taso/pk.php?otteluid=716369',
  'match_external_id': '',
  'age_group': '',
  'aggregate_match': '',
  'competition_id': 'sb2023',
  'competition_name': 'Valtakunnalliset sarjat 2023-24',
  'competition_result_service_name': '',
  'competition_logo': '',
  'competition_status': 'published',
  'competition_officiality': 'official',
  'season_id': '2023-2024',
  'organiser': 'ssbl',
  'organiser_private': '0',
  'category_id': '444',
  'category_external_id': '',
  'category_abbrevation': '',
  'category_name': 'Miesten Divari',
  'category_logo': 'https://cdn.torneopal.net/img/salibandy/sarjalogot/inssidivari_vaaka_lila.png',
  'category_logo_dark_bg': '',
  'category_group': '101',
  'category_group_name': 'Miehet',
  '

In [24]:
selected_columns = ['match_id','match_number','category_name','date','time','team_A_id','team_A_name','team_B_id','team_B_name','status']
matches = matches[selected_columns].copy()
matches.head(10)

,match_id,match_number,category_name,date,time,team_A_id,team_A_name,team_B_id,team_B_name,status
0,715140,1359,F-liiga miehet,2023-09-15,18:30:00,2478,Happee,6204,LASB,Played
1,715139,1360,F-liiga miehet,2023-09-16,17:00:00,2628,Classic,2734,Indians,Played
2,715138,1361,F-liiga miehet,2023-09-16,17:00:00,2677,SPV,2702,TPS,Played
3,715137,1362,F-liiga miehet,2023-09-16,18:00:00,2588,OLS,2576,Nokian KrP,Played
4,715136,1363,F-liiga miehet,2023-09-17,17:00:00,6041,Jymy,2830,FBC Turku,Played
5,715135,1364,F-liiga miehet,2023-09-17,17:00:00,2455,Oilers,6212,EräViikingit,Played
6,715134,1365,F-liiga miehet,2023-09-18,18:30:00,2576,Nokian KrP,2455,Oilers,Played
7,715133,1366,F-liiga miehet,2023-09-20,18:30:00,6041,Jymy,2478,Happee,Played
8,715132,1367,F-liiga miehet,2023-09-20,18:30:00,2702,TPS,2628,Classic,Played
9,715130,1369,F-liiga miehet,2023-09-22,18:30:00,2576,Nokian KrP,6212,EräViikingit,Played
